<h1>Segmenting and Clustering Neighborhoods in Toronto</h1>

 Steps 1:scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

In [1]:
import urllib.request
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
page = urllib.request.urlopen(url)

In [2]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(page, "lxml")

In [23]:
#table inside html
all_tables=soup.find_all("table")
right_table=soup.find('table', class_='wikitable sortable')

<h3>Creating table</h3>

In [5]:
A=[]
B=[]
C=[]

for row in right_table.findAll('tr'):
    cells=row.findAll('td')
    if len(cells)==3:
        A.append(cells[0].find(text=True))
        B.append(cells[1].find(text=True))
        C.append(cells[2].find(text=True))
import pandas as pd
df=pd.DataFrame(A,columns=['PostalCode'])
df['Borough']=B
df['Neighborhood']=C
df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,
1,M2A,Not assigned,
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [6]:
#removing borough contains "Not assigned" value
df=df[~df.Borough.str.contains("Not assigned")]
df.head()

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [7]:
#checking for null value
df.isnull().values.any()

False

<h1>Anwser 1</h1>

In [8]:
#reset null value
df.reset_index(drop=True, inplace=True)
df.head(10)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


<h3>Shape of dataframe created from url</h3>

In [9]:
df.shape

(103, 3)

In [10]:
#!conda install -c conda-forge geopy --yes 

Geting  Geospatial_data.csv

In [11]:
#!wget -q -O 'Geospatial_data.csv' https://cocl.us/Geospatial_data
    
df_map = pd.read_csv('Geospatial_data.csv')
    
df_map.head()


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


replacing \n value inside the dataframe from url 

In [12]:
print (df_map.dtypes)
print(df.dtypes)
#df.sort_values(by=['PostalCode'], inplace=True)
#df_map.sort_values(by=['Postal Code'], inplace=True)
#df.reset_index(drop=True, inplace=True)
#df_map.reset_index(drop=True, inplace=True)
#print (df['Postal Code'][0]==df['Postal Code'][0])
#df['Postal Code'][0]
df = df.replace('\n','', regex=True)

Postal Code     object
Latitude       float64
Longitude      float64
dtype: object
PostalCode      object
Borough         object
Neighborhood    object
dtype: object


<h1>Answer 2 </h1>

In [18]:
df_merge=pd.merge(df, df_map,left_on='PostalCode',right_on='Postal Code', how='left')
del df_merge['Postal Code']
df_merge.head(10)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


Getiing rows borough containing Toronto

In [19]:
Toronto = df_merge[df_merge['Borough'].str.contains(r'Toronto', na=True)].reset_index(drop=True)
Toronto.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M4E,East Toronto,The Beaches,43.676357,-79.293031


In [20]:
Toronto.shape

(39, 5)

In [21]:
#!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

<h1>Clustering</h1>
    clustering based on value of Borough in Torento

In [24]:
map_Toronto = folium.Map(location=[43.679563, -79.377529], zoom_start=12)

lbls=Toronto['Borough'].unique() 
colours=['red','green','blue','black']
for i in range(4):
    t=Toronto[Toronto['Borough'].str.contains(lbls[i], na=True)].reset_index(drop=True)

    # add markers to map
    for lat, lng, borough, neighborhood in zip(t['Latitude'], t['Longitude'], t['Borough'], t['Neighborhood']):
        label = '{}, {}'.format(neighborhood, borough)
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color=colours[i],
            fill=True,
            fill_color=colours[i],
            fill_opacity=0.7,
            parse_html=False).add_to(map_Toronto)  
map_Toronto        